In [68]:
import numpy as np
from matplotlib import pyplot as plt
from netCDF4 import Dataset
import matplotlib
import matplotlib.colors as colors
import metpy.calc as mpc
import xarray as xr
from utils import *
import cartopy.crs as ccrs
import imageio
import os

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
# Parameters from the simulation:
rot = False
u0 = 10
T0 = 288
h_val = 1500
a_val = 500
small_earth_fact = 20.0
cor = 7.29211e-5 * small_earth_fact
# vortex shedding mountain - modify for other test cases
mountain_center_lat = 20 # deg
mountain_center_lon = 90 # deg
mountain_radius = 12.5 # km
mountain_height = 1.5 # km

In [4]:
# Information about the paths
model = 'SE' # or 'FV3' or 'MPAS'
vert = 'hydrostatic' # or 'nonhydrostatic'

# Name of your CAM clone:
CAM_dirname = 'CAM_6_4_060_06032025'

# Edit this to your own case name
case = 'cam_6_4_060_horiz_mount_flow_se_ne60_L30km_L38'

# Modify the following according to your naming convention
if rot:
    nc_file = case + f'.cam.h0i.0001-01-01-00000_vortex_with_rot_RF_15km_tau_1_10th_day.nc'
else:
    nc_file = case + f'.cam.h0i.0001-01-01-00000_vortex_omega0_RF_15km_tau_1_10th_day.nc'

# Define the base root to the data, add your username
username = 'timand'
run_base = f'/glade/derecho/scratch/{username}/'
output_base = f'/glade/u/home/achen/'

# Automatically set some paths
run_path = run_base + case + '/run/' + nc_file
output_dir = output_base + 'plotting/'

In [6]:
nc = Dataset(run_path)
ds = xr.open_dataset(run_path)

In [8]:
# Specify the altitude, field and times you want to plot
z_val = 300 # in metres, over 300
field = 'vor' # modify this! options = 'U', 'V', 'w', 'T', 'vor', 'div'
mountain_observed_radius = mountain_radius*np.sqrt(-np.log((z_val/1000.0)/mountain_height))
mountain_radius_degrees = mountain_observed_radius / (6371.0/small_earth_fact) * 180.0/np.pi

In [9]:
# Extract the data
time_stamps = nc['time'][:]
lats = nc['lat'][:]
lons = nc['lon'][:]
levs = nc['lev'][:]

print(f'There are {len(time_stamps)} time samples')
print(f'{len(lats)} latitiude points, {len(lons)} longitude points')

There are 81 time samples
513 latitiude points, 1024 longitude points


In [10]:
# Crop to the subdomain of interest

lon_crop_left = 70.
lon_crop_right = 250.
lon_ticks = np.linspace(80, 220, 8) # Plotting ticks

lat_crop_up = 40.
lat_crop_down = 0.
lat_ticks = np.linspace(0,40,5) # Plotting ticks


lon_inds = np.where((lons > lon_crop_left) & (lons < lon_crop_right))[0]
lat_inds = np.where((lats > lat_crop_down) & (lats < lat_crop_up))[0]

print(f'Crop is of size, {len(lat_inds)} lat vals and {len(lon_inds)} lon vals')

lat_slice = lats[lat_inds] 
lon_slice = lons[lon_inds] 
LON_subset, LAT_subset = np.meshgrid(lon_slice, lat_slice)

Crop is of size, 113 lat vals and 512 lon vals


In [65]:
if field == 'T':
    title = 'Temperature perturbation'
    cb_label = '$T - T_0$ [K]'
    min = -2
    max = 5
elif field == 'U':
    title = 'Normalised zonal velocity perturbation'
    cb_label = '$(u(t) - u_0)/u_0$ [m/s]'
    min = -1.5
    max = 0.6
elif field == 'V':
    title = "Meridional velocity"
    cb_label = '$v(t)$ [m/s]'
    min = -9
    max = 9
elif field == 'w':
    title = "Vertical velocity"
    cb_label = '$w(t)$ [m/s]'
    min = -0.2
    max = 0.4
elif field == "vor":
    title = "Relative vorticity"
    cb_label = '$\zeta(t)$ [1/s]'
    round_dig = 6
    min = -0.0001
    max = 0.0001
elif field == "div":
    title = "Fluid divergence"
    cb_label = '$\sigma(t)$ [1/s]'
    round_dig = 6
    min = -0.00003
    max = 0.00003

if ((-1e-10 > min) and (1e-10 < max)):
    cmap_choice = 'RdBu_r'
    norm = MidpointNormalize(midpoint = 0, vmin = min, vmax = max)
elif (min < 1e-10):
    cmap_choice = "Blues"   
    norm = colors.Normalize(vmin=min, vmax=max)
else:
    cmap_choice = 'RdYlBu_r'
    norm = colors.Normalize(vmin=min, vmax=max)

levels = 11
levels = np.mgrid[min:max:levels*1j]

In [81]:
individual_plots = []

for i in range(np.size(time_stamps)):
# for i in range(1, 2):
    print(i)
    # interpolate the data at that time step
    if (model == "MPAS"): # directly interp in z
        if (field == "vor" or field == "div"):
            uv = ds[['U','V']].isel(time=i, lat=lat_inds, lon=lon_inds).interp(lev=300, method='linear')
            if (field == "vor"):
                data = mpc.vorticity(ds['U'], ds['V'])
            else:
                data = mpc.divergence(ds['U'], ds['V'])
        else:
            data = ds[field].isel(time=i,lat=lat_inds, lon=lon_inds).interp(lev=300, method='linear')
    else:
        if (field == "vor" or field == "div"):
            uv = np.zeros((2, len(lat_slice), len(lon_slice)))
            uv = z_interp_uv(nc['Z3'][i, :, lat_inds, lon_inds], nc['U'][i, :, lat_inds, lon_inds], nc['V'][i, :, lat_inds, lon_inds], lon_slice, lat_slice, z_val)
            uv_ds = xr.Dataset(data_vars = dict(U=(['lat', 'lon'], uv[0]), V=(['lat', 'lon'], uv[1])), coords = dict(lat=lat_slice, lon=lon_slice))
            uv_ds['U'].attrs["units"] = "m/s"
            uv_ds['V'].attrs["units"] = "m/s"
            if (field == "vor"):
                data = mpc.vorticity(uv_ds['U'], uv_ds['V'])
            else:
                data = mpc.divergence(uv_ds['U'], uv_ds['V'])
        elif (field == "w"):
            if (vert == 'hydrostatic'):
                data = z_interp_w_hydrostatic(nc, t_idxs[i], lon_inds, lat_inds, z_val)
            else: 
                data = z_interp_w_nonhydro(nc, t_idxs[i], lon_inds, lat_inds, z_val)
        else:
            data = z_interp(nc['Z3'][i, :, lat_inds, lon_inds], nc[field][i, :, lat_inds, lon_inds], lon_slice, lat_slice, z_val)
    # some post processing
    if (field == "U"):
        LAT_subset_rad = np.deg2rad(LAT_subset)
        data = (data - u0*np.cos(LAT_subset_rad)) / (u0*np.cos(LAT_subset_rad))
    elif (field == "T"):
        data = data - T0
    # plot the data at that time step

    fig = plt.figure(figsize=(10, 5))
    # ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax = fig.add_subplot(1, 1, 1)
    title_size=16
    label_size=14
    small_size=12
    round_dig = 2
    
    plot = ax.contourf(lon_slice, lat_slice, data, levels=levels, cmap=cmap_choice, norm=norm, extend='both')

    ax.tick_params(axis='both', labelsize=small_size)
    ax.set_title(f'Vortex shedding test case, {z_val} meters, \n' + '%.*f' % (4, time_stamps[i]) + ' days, min = ' + '%.*f' % (round_dig, np.nanmin(data)) + ' max = ' + '%.*f' % (round_dig, np.nanmax(data)))
    ax.add_patch(plt.Circle((mountain_center_lon, mountain_center_lat), 1.2*mountain_radius_degrees, color='tab:brown'))

    cb = plt.colorbar(plot, ax=ax, fraction=0.05, location='bottom')
    cb.set_label(label=title + " " + cb_label, size=small_size)
    cb.ax.tick_params(labelsize=small_size)
    ax.set_aspect('equal')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    name = str(i) + '.png'
    plt.savefig(name, dpi=300, bbox_inches='tight')
    individual_plots.append(name)
    plt.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [82]:
w = imageio.get_writer(f'{output_dir}/vortex_shedding.gif')
for filename in individual_plots:
    image = imageio.imread(filename)
    w.append_data(image)
w.close()

/glade/derecho/scratch/achen/tmp/ipykernel_73784/4074246050.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [83]:
for filename in set(individual_plots):
    os.remove(filename)